In [15]:
import tensorflow as tf
from flask import Flask, request
import numpy as np
import mediapipe as mp
import cv2
import base64

In [16]:
model = tf.keras.models.load_model('my_model.keras')

In [17]:
actions = np.array(['hola', 'ok', 'a'])

In [18]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [19]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [20]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [21]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    right = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    left = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    return np.concatenate([pose, face, right, left])

In [22]:
def decode_image(img):
    image_data = img.split(",")[1]
    image_bytes = base64.b64decode(image_data)
    image_buffer = np.frombuffer(image_bytes, dtype=np.uint8)
    frame = cv2.imdecode(image_buffer, cv2.IMREAD_COLOR)
    return frame

In [23]:
holistic =  mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [25]:
from flask import jsonify, render_template
from flask_socketio import SocketIO, send

from datetime import datetime


In [30]:
app = Flask(__name__)
socketio = SocketIO(app)
socketio.init_app(app, cors_allowed_origins="*")


frames = []
sequence = []
predictions = []
threshold = 0.5

@socketio.on("connect")
def on_connect():
    print("Cliente conectado")

@app.route("/")
def index():
    return render_template("index.html")

@socketio.on("message")
def on_message(data):   
   
    global sequence
    #print(data["frames"])
    image_base64 = data["frames"]
    start_time = datetime.now()
    frame = decode_image(image_base64)
    end_time = datetime.now()
    print(end_time - start_time)
    
    image, results = mediapipe_detection(frame, holistic)
    keypoints = extract_keypoints(results)
    print(keypoints)
    sequence.append(keypoints)
    
    if len(sequence) == 30:
        #print(sequence[1])
        prediction = "aqui"
        socketio.emit("prediction", {"prediction": prediction})
        sequence = []


    #data = request.get_json()
    
    #frames = data["frames"]
    #frames
    #for frame in frames:
    #    image, result = mediapipe_detection(frame, holistic)
    #    keypoints = extract_keypoints(result)
    #    sequence.append(keypoints)
    #    #print(sequence)
    #sequence = sequence[-30:]
    #
    #res = model.predict(np.expand_dims(sequence, axis=0))[0]    
    #prediction = actions[np.argmax(res)]
    #print(prediction)

    #prediction = "aaaaaa"

    #socketio.emit("prediction", {"prediction": prediction})

if __name__ == "__main__":
    socketio.run(app, host="localhost", port=5000, allow_unsafe_werkzeug=True)

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit


127.0.0.1 - - [04/Apr/2024 23:55:32] "GET /socket.io/?EIO=4&transport=polling&t=OwidAZn HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2024 23:55:32] "POST /socket.io/?EIO=4&transport=polling&t=OwidAej&sid=sefgwRKmWVX9nohOAAAA HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2024 23:55:32] "GET /socket.io/?EIO=4&transport=polling&t=OwidAel&sid=sefgwRKmWVX9nohOAAAA HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2024 23:55:32] "GET /socket.io/?EIO=4&transport=polling&t=OwidAjl&sid=sefgwRKmWVX9nohOAAAA HTTP/1.1" 200 -


Cliente conectado
0:00:00.003806
[ 0.38667589  0.45222715 -1.07897031 ...  0.          0.
  0.        ]
0:00:00.002997
[ 0.38688821  0.46554831 -1.08600736 ...  0.          0.
  0.        ]
0:00:00.001999
[ 0.3867391   0.47395614 -1.11102057 ...  0.          0.
  0.        ]
0:00:00.002998
[ 0.38563144  0.47422346 -1.09637487 ...  0.          0.
  0.        ]
0:00:00.003000
[ 0.38562304  0.47466558 -1.11898923 ...  0.          0.
  0.        ]
0:00:00.003750
[ 0.38522202  0.47443005 -1.1105938  ...  0.          0.
  0.        ]
0:00:00.003000
[ 0.39032888  0.47416499 -1.09846985 ...  0.          0.
  0.        ]
0:00:00.002998
[ 0.38972086  0.47359869 -1.07965171 ...  0.          0.
  0.        ]
0:00:00.003002
[ 0.38937005  0.47329894 -1.08379912 ...  0.          0.
  0.        ]
0:00:00.002999
[ 0.38905042  0.47326988 -1.08137846 ...  0.          0.
  0.        ]
0:00:00.002998
[ 0.38916999  0.47238714 -1.08047795 ...  0.          0.
  0.        ]
0:00:00.002997
[ 0.39022034  0.47077

127.0.0.1 - - [04/Apr/2024 23:55:44] "GET /socket.io/?EIO=4&transport=websocket&sid=sefgwRKmWVX9nohOAAAA HTTP/1.1" 200 -


[ 0.44552723  0.4342677  -1.04118025 ...  0.          0.
  0.        ]
0:00:00.005999
[ 0.44605812  0.43419537 -1.202142   ...  0.          0.
  0.        ]
